# EEG - Flow

## 2. Annotate bad segments and autoselect bad channels

The bad channels and the bad segments must be annotated and excluded from subsequent analysis. 

In [2]:
from pathlib import Path
from itertools import chain

from mne.io import read_raw_fif, write_info
from mne.preprocessing import compute_bridged_electrodes, interpolate_bridged_electrodes
from pyprep import NoisyChannels

from eeg_flow.config import load_config
from eeg_flow.utils.annotations import merge_bad_annotations
from eeg_flow.utils.bids import get_fname, get_folder
from eeg_flow.utils.concurrency import lock_files
from eeg_flow.viz import plot_bridged_electrodes

from mne.viz import set_browser_backend

%matplotlib qt
set_browser_backend('qt')

_, derivatives_folder, experimenter = load_config()

Using qt as 2D backend.


The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [3]:
participant = 3  # int
group =  1  # int
task = "oddball"  # str
run = 1  # int

derivatives_folder_preprocessed_p = get_folder(derivatives_folder / "preprocessed", participant, group)
derivatives_folder_plots_p = get_folder(derivatives_folder / "plots", participant, group)
fname_stem = get_fname(participant, group, task, run)

#create derivatives plots subfolder
if not Path(derivatives_folder_plots_p / fname_stem).exists():
    Path(derivatives_folder_plots_p  / fname_stem).mkdir(parents=True)
    
# create locks
derivatives = (
    derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step2_info.fif"),
    derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step2_oddball_with_bads_annot.fif"),
)
locks = lock_files(*derivatives)

# load raw recording
raw = read_raw_fif(derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step1_raw.fif"), preload=True)

Opening raw data file L:\EEG_Flow_data\derivatives\preprocessed\sub-P03-G1\sub-P03-G1_task-oddball_run-1\sub-P03-G1_task-oddball_run-1_step1_raw.fif...
Isotrak not found
    Range : 6216 ... 384492 =      6.070 ...   375.480 secs
Ready.
Reading 0 ... 378276  =      0.000 ...   369.410 secs...


## 2.1 Bridges

More information can be found on this [MNE tutorial](https://mne.tools/stable/auto_examples/preprocessing/eeg_bridging.html).

In [4]:
fig, ax = plot_bridged_electrodes(raw)
fig.suptitle(fname_stem, fontsize=16,y=1.0)

fname = derivatives_folder_plots_p / fname_stem / (fname_stem + "_0_bridges.png")
fig.savefig(fname)

Local minimum 2.7016422638212756 found
Bridge detected between F4 and F2
Bridge detected between PO5 and PO3
Bridge detected between PO5 and PO7
Bridge detected between PO3 and PO7
Bridge detected between PO4 and PO6


If the bridges don't look fixable, the recording should be probably dropped. Else, they are interpolated:

In [5]:
raw.set_montage("standard_1020")  # we need a montage for the interpolation
bridged_idx, _ = compute_bridged_electrodes(raw)
try:
    raw = interpolate_bridged_electrodes(raw, bridged_idx)
except RuntimeError:
    bads_idx = sorted(set(chain(*bridged_idx)))
    raw.info["bads"] = [raw.ch_names[k] for k in bads_idx]
    assert "M1" not in raw.info["bads"]
    assert "M2" not in raw.info["bads"]

Local minimum 2.7016422638212756 found
Bridge detected between F4 and F2
Bridge detected between PO5 and PO3
Bridge detected between PO5 and PO7
Bridge detected between PO3 and PO7
Bridge detected between PO4 and PO6
Creating RawArray with float64 data, n_channels=1, n_times=378277
    Range : 6216 ... 384492 =      6.070 ...   375.480 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=378277
    Range : 6216 ... 384492 =      6.070 ...   375.480 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=378277
    Range : 6216 ... 384492 =      6.070 ...   375.480 secs
Ready.
Interpolating bad channels
    Automatic origin fit: head of radius 97.1 mm
Computing interpolation matrix from 59 sensor positions
Interpolating 7 sensors


## 2.2 Bad channels

Bad channels are suggested by `pyprep`. A visual inspection is however needed to confirm or modify the output from `pyprep`.

In [6]:
raw.filter(
    l_freq=1.0,
    h_freq=100.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

ns = NoisyChannels(raw, do_detrend=False)  # operates only on EEG
ns.find_bad_by_SNR()
ns.find_bad_by_correlation()
ns.find_bad_by_hfnoise()
ns.find_bad_by_nan_flat()
ns.find_bad_by_ransac()  # requires electrode position
print (ns.get_bads())

raw.info["bads"].extend(
    [ch for ch in ns.get_bads() if ch not in ("M1", "M2")]
)
raw.info["bads"] = list(set(raw.info["bads"]))

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-12 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-12 dB cutoff frequency: 112.50 Hz)
- Filter length: 3381 samples (3.302 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


Executing RANSAC
This may take a while, so be patient...
Progress: 10%... 20%... 30%... 40%... 50%... 60%... 70%... 80%... 90%... 100%

RANSAC done!
['FC6']


## 2.3 Visual inspection and annotations of bad segments

During the visual inspection, bad segments should be annotated.

In [7]:
raw.filter(
    l_freq=1.0,
    h_freq=40.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-12 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-12 dB cutoff frequency: 45.00 Hz)
- Filter length: 3381 samples (3.302 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


<Raw | sub-P03-G1_task-oddball_run-1_step1_raw.fif, 68 x 378277 (369.4 s), ~196.3 MB, data loaded>

In [8]:
raw.plot(theme="light")

2023-04-06 19:41:12,837 - qdarkstyle - INFO - QSS file successfully loaded.
2023-04-06 19:41:12,838 - qdarkstyle - INFO - Found version patches to be applied.
2023-04-06 19:41:12,839 - qdarkstyle - INFO - Found application patches to be applied.


The segments annotated as bads are mapped onto the oddball events, to standardize the annotation procedure and to mark only the signal used in the `ICA` and in the `Epochs` as bad.

In [ ]:
annotations = merge_bad_annotations(raw)

## 2.4 Save derivatives

The updated annotations can now be saved alongside the selected bad channels.

In [ ]:
fname_info = derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step2_info.fif")
assert not fname_info.exists()  # write_info always overwrites 
write_info(fname_info, raw.info)
fname_annot = derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step2_oddball_with_bads_annot.fif")
annotations.save(fname_annot, overwrite=False)

Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway